# Dv and travel time computation from models

This notebook demonstrates the generation of moon models and their corresponding dv and travel times:
- Creating Moon models and mesh (M1, M2, M3, weber_core_smooth, weber_core)
- Sources, receivers, and phases with rays computed
- Computation and visualization of sensitivity kernels with sparse G matrix calculation
- Delta V scalar field generation, projection, and visualisation
- Computation of travel times using dv with G

In [1]:
import numpy as np
from sensray import PlanetModel
from fwdop import GFwdOp, make_scalar_field

## 1. Creation of Moon model and mesh

Creating a model and mesh for ray tracing

### Creating model

Creating a moon model from information stored in .nd file
- User-defined model (model_name), mesh size, and inner core mesh size
- Model creation of PlanetModel class

In [2]:
# Load model and create mesh
model_name = "M1"
mesh_size_km = 1000

# Create mesh and save if not exist, otherwise load existing
mesh_path = "M1_mesh"

# Load model and create mesh
model = PlanetModel.from_standard_model('M1')

### Creating layered tetrahedral mesh

Uses a layered tetrahedral mesh to control resolution across discontinuities
- User-defined mesh name
- Radii and H_layers gathered from the model properties
- Populate mesh properties with vp, vs, rho from the file (also gathered from model)

In [3]:
try:
    model.create_mesh(from_file=mesh_path)
    print(f"Loaded existing mesh from {mesh_path}")
except FileNotFoundError:
    print("Creating new mesh...")
    radii = model.get_discontinuities()
    H_layers = [1000, 600]
    model.create_mesh(mesh_size_km=mesh_size_km, radii=radii, H_layers=H_layers)
    model.mesh.populate_properties(model.get_info()["properties"])
    model.mesh.save(f"{model_name}_mesh")  # Save mesh to VT
print(f"Created mesh: {model.mesh.mesh.n_cells} cells")

Loaded mesh from M1_mesh.vtu
Loaded metadata: 9975 cells, 2258 points
Loaded existing mesh from M1_mesh
Created mesh: 9975 cells


## 2. Define Source-Receiver Geometry

Set up realistic earthquake-station-phase and translate to earthquake-station-ray combination.

### Point generation

Using predefined single source-receiver-phase to generate a ray

In [4]:
# testing with one source-receiver pair - same as initial test
source_lat, source_lon, source_depth = 0.0, 0.0, 150.0  # Equator, 150 km depth
receiver_lat, receiver_lon = 30.0, 45.0  # Surface station
srp = [((source_lat, source_lon, source_depth), (receiver_lat, receiver_lon), ["P", "S", "ScS"])]

### Ray tracing with taup

Function to generate a ray using taup for each phase of interest for each source-receiver pair. Save as a list of lists of [source-receiver-ray] to be passed into the forward operatot

In [5]:
def get_rays(srp):
    '''
    srp: list of tuples (source, receiver, phases)
    where source = (lat, lon, depth), receiver = (lat, lon), phases = [phase1, phase2, ...]
    returns array of (source, receiver, ray) for each ray
    '''
    srr_list = []
    for (source, receiver, phases) in srp:
        rays = model.taupy_model.get_ray_paths_geo(
            source_depth_in_km=source[2],
            source_latitude_in_deg=source[0],
            source_longitude_in_deg=source[1],
            receiver_latitude_in_deg=receiver[0],
            receiver_longitude_in_deg=receiver[1],
            phase_list=phases,
        )
        for ray in rays:
            srr_list.append((source, receiver, ray))

    return np.array(srr_list, dtype=object)


print("Compute ray for each source-receiver-phase combination...")
srr = get_rays(srp)

Compute ray for each source-receiver-phase combination...
Building obspy.taup model for '/home/matth/Masters-Project/SensRay/sensray/models/M1.nd' ...
filename = /home/matth/Masters-Project/SensRay/sensray/models/M1.nd
Done reading velocity model.
Radius of model . is 1737.1
Using parameters provided in TauP_config.ini (or defaults if not) to call SlownessModel...
Parameters are:
taup.create.min_delta_p = 0.1 sec / radian
taup.create.max_delta_p = 11.0 sec / radian
taup.create.max_depth_interval = 115.0 kilometers
taup.create.max_range_interval = 0.04363323129985824 degrees
taup.create.max_interp_error = 0.05 seconds
taup.create.allow_inner_core_s = True
Slow model  643 P layers,747 S layers
Done calculating Tau branches.
Done Saving /tmp/M1.npz
Method run is done, but not necessarily successful.


## 3. Compute sensitivity kernels and G matrix

Compute travel-time sensitivity kernels: K = -L / v² for each ray using GFwdOp(LinearOperator) and store as a sparse G matrix

In [6]:
print("Calculate sensitivity kernels and sparse G matrix...")
G = GFwdOp(model, srr[:,2])

Calculate sensitivity kernels and sparse G matrix...
Stored sensitivity kernel as cell data: 'K_P_vp'
Stored sensitivity kernel as cell data: 'K_S_vs'
Stored sensitivity kernel as cell data: 'K_ScS_vs'
Stored sensitivity kernel as cell data: 'K_S_vs'
Stored sensitivity kernel as cell data: 'K_ScS_vs'
P Kernel Matrix shape: (1, 9975), nnz: 24
S Kernel Matrix shape: (2, 9975), nnz: 59
ScS Kernel Matrix shape: (2, 9975), nnz: 122


# 4. Create Dv and apply to G

Create vectorized scalar field from radial and angular functions R(r), T(θ,φ) then project onto the mesh. Apply Dv to G to compute travel times. Choice of functions to use

### Scalar field generation

Import function to convert radial and angular functions to a vectorized scalar function in f(x,y,z)

In [7]:
# Generating a scalar field from functions in R, T
print(f"Scalar field generation...")

functions = {
    "simple": {"R": lambda r: np.ones_like(r), "T": lambda theta, phi: np.ones_like(theta)},
    "complex": {"R": lambda r: r**2 * np.exp(-r/100000), "T": lambda theta, phi: np.cos(theta)},
    "harmonic": {"R": lambda r: 0.1 * model.get_property_at_radius(radius=r, property_name="vp"), "T": lambda theta, phi: 0.5 * np.sqrt(5 / np.pi) * (3 * np.cos(theta) ** 2 - 1)},
}

func = "harmonic"
f = make_scalar_field(functions[func]["R"], functions[func]["T"])

Scalar field generation...


### Dv mesh projection and visualisation

Project scalar field onto the mesh to calculate dv

In [8]:
print("Dv projection onto mesh...")
model.mesh.project_function_on_mesh(f, property_name="dv")

Dv projection onto mesh...


Visualise dv field using wrapper in mesh and first source-receiver pair for calculation of plane for slice

In [9]:
print("Dv visualization...")
model.mesh.display_dv(srr[0,0][0], srr[0,0][1], srr[0,1][0], srr[0,1][1], property_name="dv")

Dv visualization...
Background P-wave velocity:


Widget(value='<iframe src="http://localhost:43473/index.html?ui=P_0x7c04a246ed10_0&reconnect=auto" class="pyvi…

### Time travel computation

Apply dv to G forward operator to compute travel times.

In [10]:
print("Time travel computation...")
travel_times = G(model.mesh.mesh.cell_data["dv"])
print(travel_times)

Time travel computation...
[ 8.34158499 25.28607367 43.01808839 89.37476113 57.58353932]
